In [1]:
pip install tensorflow_text

     |████████████████████████████████| 8.3MB 28kB/s 
     |████████████████████████████████| 421.8MB 35kB/s 
     |████████████████████████████████| 450kB 35.7MB/s 
     |████████████████████████████████| 3.9MB 44.2MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import re

In [3]:
df1 = pd.read_csv('GoodFile1.csv', sep=';')

df1["review"] = df1["Review"]
df1["type"] = df1["review_type"]

df = df1[["review", "type"]]
print(df)

good_reviews = df[df.type == "good"]
bad_reviews = df[df.type == "bad"]

good_reviews_text = " ".join(good_reviews.review.to_numpy().tolist())
bad_reviews_text = " ".join(bad_reviews.review.to_numpy().tolist())

good_df = good_reviews.sample(n=len(bad_reviews), random_state=random.seed())
bad_df = bad_reviews
review_df = good_df.append(bad_df).reset_index(drop=True)
print(review_df.review)

                                                  review  type
0      да все таки он немного похож на него но мой ма...  good
1                  rt ну ты идиотка я испугалась за тебя  good
2      rt кто то в углу сидит и погибает от голода а ...  good
3      вот что значит страшилка d но блин посмотрев в...  good
4               ну любишь или нет я не знаю кто ты бля d  good
...                                                  ...   ...
65529               прасти но сигодня тожи ни приду 9 69   bad
65530  rt я вдруг сегодня понял что птицам на меня на...   bad
65531                                   и так каждый раз   bad
65532  ну что за лядство неделю по крупицам восстанав...   bad
65533  здравствуйте фото не открывается подскажите ка...   bad

[65534 rows x 2 columns]
0        пол шестого утра я вот думаю ложиться спать см...
1        думала что же написать и вспомнила по дороге д...
2        приглядела тут звуковую карточку взамен встрое...
3                             rt как деак

In [5]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

from sklearn.preprocessing import OneHotEncoder

type_one_hot = OneHotEncoder(sparse=False).fit_transform(
  review_df.type.to_numpy().reshape(-1, 1)
)

train_reviews, test_reviews, y_train, y_test =\
  train_test_split(
    review_df.review,
    type_one_hot,
    test_size=.1,
    random_state=random.seed()
  )

print("start to train")

X_train = []
for r in tqdm(train_reviews):
  emb = embed(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_train.append(review_emb)

X_train = np.array(X_train)

X_test = []
for r in tqdm(test_reviews):
  emb = embed(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_test.append(review_emb)

X_test = np.array(X_test)

print(X_train.shape, y_train.shape)

  0%|          | 0/58980 [00:00<?, ?it/s]

start to train


100%|██████████| 6554/6554 [06:08<00:00, 17.79it/s]

(58980, 512) (58980, 2)


In [6]:
pip install keras

In [7]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
model = tf.keras.Sequential()

model.add(
  tf.keras.layers.Dense(
    units=256,
    input_shape=(X_train.shape[1], ),
    activation='relu'
  )
)
model.add(
  tf.keras.layers.Dropout(rate=0.5)
)

model.add(
  tf.keras.layers.Dense(
    units=128,
    activation='relu'
  )
)
model.add(
  tf.keras.layers.Dropout(rate=0.5)
)

model.add(tf.keras.layers.Dense(2, activation='softmax'))
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    shuffle=True
)

model.evaluate(X_test, y_test)

Train on 53082 samples, validate on 5898 samples
Epoch 1/10
53082/53082 [==============================] - 12s 223us/sample - loss: 0.5451 - accuracy: 0.7207 - val_loss: 0.5105 - val_accuracy: 0.7397
Epoch 2/10
53082/53082 [==============================] - 11s 213us/sample - loss: 0.5129 - accuracy: 0.7452 - val_loss: 0.5000 - val_accuracy: 0.7523
Epoch 3/10
53082/53082 [==============================] - 11s 215us/sample - loss: 0.4885 - accuracy: 0.7624 - val_loss: 0.4914 - val_accuracy: 0.7538
Epoch 4/10
53082/53082 [==============================] - 11s 215us/sample - loss: 0.4686 - accuracy: 0.7739 - val_loss: 0.4865 - val_accuracy: 0.7589
Epoch 5/10
53082/53082 [==============================] - 11s 214us/sample - loss: 0.4474 - accuracy: 0.7889 - val_loss: 0.4873 - val_accuracy: 0.7616
Epoch 6/10
53082/53082 [==============================] - 11s 216us/sample - loss: 0.4272 - accuracy: 0.8004 - val_loss: 0.4878 - val_accuracy: 0.7603
Epoch 7/10
53082/53082 [=====================

[0.5249265961105736, 0.752365]

In [0]:
model_json = model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model1.h5")

In [10]:
sentence = ["отличный день"]

result = embed(sentence)
print(model.predict(np.array(result)))

[[0.16588685 0.8341132 ]]


In [13]:
sentence = ["плохой день"]

result = embed(sentence)
print(model.predict(np.array(result)))

[[0.99535036 0.00464962]]
